In [ ]:
!git clone https://github.com/syncdoth/Eagle-Eye-Pneumonia-Detection.git

fatal: not in a git directory
fatal: not in a git directory
Cloning into 'Eagle-Eye-Pneumonia-Detection'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 172 (delta 81), reused 115 (delta 46), pack-reused 0
Receiving objects: 100% (172/172), 122.65 KiB | 6.13 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
!bash Eagle-Eye-Pneumonia-Detection/utils/connect_in_colab.sh

The following NEW packages will be installed:
  sshpass
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 10.5 kB of archives.
After this operation, 30.7 kB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package sshpass.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../sshpass_1.06-1_amd64.deb ...
Unpacking sshpass (1.06-1) ...
Setting up sshpass (1.06-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
┌━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd Eagle-Eye-Pneumonia-Detection

/content/Eagle-Eye-Pneumonia-Detection


In [ ]:
!pip install --upgrade pip
!pip install -r colab_requirements.txt

     |████████████████████████████████| 1.5MB 8.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 1.9 MB 63.9 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16033 sha256=57fb710c9d6fea7fbba8ed52df3155a5da6169e00f68ae55331183c95c0d5270
  Stored in directory: /root/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import torch
from importlib import reload
from torch.utils.data import SubsetRandomSampler
from models.type_classifier import TypeClassifier
import models.train
reload(models.train)
from models.train import train, make_batch
import dataloader.ClassificationDataset
reload(dataloader.ClassificationDataset)
from dataloader.ClassificationDataset import ClassificationDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset = ClassificationDataset(root="/content/data_server")

train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, collate_fn=make_batch, sampler=SubsetRandomSampler(dataset.train_idx))
val_loader = torch.utils.data.DataLoader(dataset, batch_size=32, collate_fn=make_batch, sampler=SubsetRandomSampler(dataset.val_idx))
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32, collate_fn=make_batch, sampler=SubsetRandomSampler(dataset.test_idx))

In [ ]:
pretrain_model = TypeClassifier(version="b0", num_classes=4).to(device)

optimizer = torch.optim.Adam(pretrain_model.parameters(), lr=1e-5)
epochs = 1

train(pretrain_model, optimizer, train_loader, device, torch.nn.CrossEntropyLoss(), val_dataset=val_loader, epochs=epochs)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


current loss: 1.3282341957092285, current f1_score: 0.11904761904761904 /current loss: 1.3117247819900513, current f1_score: 0.16666666666666669 /current loss: 1.3255523443222046, current f1_score: 0.1836734693877551 /current loss: 1.2982178926467896, current f1_score: 0.16666666666666666 /

RuntimeError: ignored